<a href="https://colab.research.google.com/github/ilham-mukti/Machine-Learning/blob/main/Workflow_Pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Prep

In [ ]:
#1 ini dataset bawaan pytorch
trainset = datasets.CIFAR10('/content/cifar2/', download=True, train=True, transform=train_transform)
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)

testset = datasets.CIFAR10('/content/cifar2/', download=True, train=False, transform=test_transform)
testloader = DataLoader(testset, batch_size=bs, shuffle=True, num_workers=2)

In [ ]:
#2 Multiclass
# /data/train/kue_dadar/1.jpg
# /data/train/kue_salju/1.jpg

# /data/test/kue_dadar/1.jpg

# referensi: https://www.kaggle.com/ilhammukti/cnn-transfer-learning-mobilenetv2-kue-indonesia
# referensi: data hama

train_set = datasets.ImageFolder("../input/kue-indonesia/train/", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True)

val_set = datasets.ImageFolder("../input/kue-indonesia/validation/", transform=test_transform)
valloader = DataLoader(val_set, batch_size=bs, shuffle=True)

test_set = datasets.ImageFolder("../input/kue-indonesia/test/", transform=test_transform)
testloader = DataLoader(test_set, shuffle=True)

In [ ]:
#3 Multiclass
# Cuma ada 1 folder, jadi kita harus split jadi data_train, val, dan test

# /data/kue_dadar/1.jpg
# /data/kue_salju/1.jpg
# referensi: https://www.kaggle.com/ilhammukti/aksara-jawa-letter-classification

PATH = '../input/hanacaraka/'
images, labels = [], []
for dirname, _, filenames in os.walk(PATH):
  for filename in filenames:
    images.append(os.path.join(dirname, filename).split('/')[-1]) # /folder/blabla/xxx.jpg
    labels.append(os.path.join(dirname, filename).split('/')[-2])

##
label2cat = np.unique(labels) #INI CEK DULU, NAMA FOLDERNYA MASUK ENGGA!

##
datas = pd.DataFrame({'image_id':images, 'label':labels})
datas.head(), datas.shape

##
index = []
for i in range(len(datas)):
    try:
        Image.open(PATH + str(datas['label'].values[i]) + '/' + str(datas['image_id'].values[i]))
        
    except PIL.UnidentifiedImageError:
        index.append(i)
  
    except FileNotFoundError:
        index.append(i)

datas = datas.drop(index)
datas.head(), datas.shape

###
X_train, X_val, y_train, y_val = train_test_split(
    datas['image_id'].values, datas['label'].values, 
    test_size=0.3, stratify=datas['label'].values, 
    random_state=24)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,
    test_size=0.5, stratify=y_val, 
    random_state=24)


len(X_train), len(X_val), len(X_test)

In [ ]:
#4 Multi label
# /data/train/x.jpg
# /data/test/y.jpg

# dilabelinnya di csv
# train_label.csv dan test_label.csv: x.jpg, terus nama kelas (one hot)

# referensi https://github.com/ilham-mukti/Mencoba-Neural-Network/blob/main/CNN_Multilabel_Transfer_Learning_FileUpload_(Data_Panorama).ipynb
train_set = MultilabelDataset("/content/data/train_label.csv", "/content/data/train/", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=2)

test_set = MultilabelDataset("/content/data/test_label.csv", "/content/data/test/", transform=test_transform)
testloader = DataLoader(test_set, batch_size=bs, shuffle=True)

# Model Arsitektur

In [ ]:
#Custom model
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
          conv_block(3, 8),
          conv_block(8, 16),
          conv_block(16, 32),
          conv_block(32, 64),
          conv_block(64, 128),
          nn.Flatten()
      )
    self.fc = nn.Sequential(
          linear_block(128*4*4, 256, dropout=0.2),
          linear_block(256, 2, activation='lsoftmax')
      )
  def forward(self, x):
    x = self.conv(x)
    x = self.fc(x)
    return x

In [ ]:
# Pake Model orang Lain
class CustomMobileNetv2(nn.Module):
  def __init__(self, output_size):
    super().__init__()
    self.mnet = mobilenet_v2(pretrained=True)
    self.freeze()

    self.mnet.classifier = nn.Sequential(
        nn.Linear(1280, output_size),
        nn.Sigmoid() #sigmoid untuk multilabel, Logsoftmax untuk multiclass
    )

  def forward(self, x):
    return self.mnet(x)
  
  def freeze(self):
    for param in self.mnet.parameters():
      param.requires_grad = False

  def unfreeze(self):
    for param in self.mnet.parameters():
      param.requires_grad = True

# Predict

In [ ]:
#multi label
# referensi: https://github.com/ilham-mukti/Mencoba-Neural-Network/blob/main/CNN_Multilabel_Transfer_Learning_FileUpload_(Data_Panorama).ipynb
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)
with torch.no_grad():
  model.eval()
  output = model(feature)
  preds = (output > 0.5).to(torch.float32)
preds

def convert_to_label(x):
  return [label for pred, label in zip(x, label2cat) if pred==1]

def inverse_norm(img):
  img[0, :, :] = img[0, :, :] * 0.229 + 0.485
  img[1, :, :] = img[1, :, :] * 0.224 + 0.456
  img[2, :, :] = img[2, :, :] * 0.225 + 0.406
  return img

fig, axes = plt.subplots(6, 6, figsize=(24, 24))
for img, label, pred, ax in zip(feature, target, preds, axes.flatten()):
  ax.imshow(inverse_norm(img).permute(1,2,0).cpu())
  font = {"color":'r'} if (pred != label).any() else {"color": 'g'}
  label, pred = convert_to_label(label), convert_to_label(pred)
  ax.set_title(f"Label: {label}\nPred: {pred}", fontdict=font);
  ax.axis("off");

In [ ]:
# Multi class
# referensi https://github.com/ilham-mukti/Mencoba-Neural-Network/blob/main/CNN_Multiclass_File_Upload_HAMA.ipynb
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)
label2cat = train_set.classes

with torch.no_grad():
  model.eval()
  output = model(feature)
  preds = output.argmax(1)
preds

fig, axes = plt.subplots(6, 6, figsize=(24, 24))
for img, label, pred, ax in zip(feature, target, preds, axes.flatten()):
  ax.imshow(img.permute(1,2,0).cpu())
  font = {"color":'r'} if label != pred else {"color": 'g'}
  label, pred = label2cat[label.item()], label2cat[pred.item()]
  ax.set_title(f"Label: {label} | Pred: {pred}", fontdict=font);
  ax.axis("off");

# Load Model

In [ ]:
weights = torch.load('/content/model/weights_best.pth', map_location="cpu")
config = torch.load('/content/model/configs.pth', map_location='cpu')

model = CNN()
model.load_state_dict(weights)
model = model.to(device)